### Try-it 9.2: Predicting Wages

This activity is meant to summarize your work with regularized regression models.  You will use your earlier work with data preparation and pipelines together with what you've learned with grid searches to determine an optimal model.  In addition to the prior strategies, this example is an excellent opportunity to utilize the `TransformedTargetRegressor` estimator in scikitlearn.

### The Data

This dataset is loaded from the openml resource library.  Originally from census data, the data contains wage and demographic information on 534 individuals. From the dataset documentation [here](https://www.openml.org/d/534)

```
The Current Population Survey (CPS) is used to supplement census information between census years. These data consist of a random sample of 534 persons from the CPS, with information on wages and other characteristics of the workers, including sex, number of years of education, years of work experience, occupational status, region of residence and union membership. 
```

In [13]:
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np
import plotly.express as px
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

In [8]:
wages = fetch_openml(data_id=534, as_frame=True)

/Applications/Anaconda/anaconda3/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [10]:
wages.frame.head()

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,WAGE,AGE,RACE,OCCUPATION,SECTOR,MARR
0,8,no,female,21,not_member,5.10,35,Hispanic,Other,Manufacturing,Married
1,9,no,female,42,not_member,4.95,57,White,Other,Manufacturing,Married
2,12,no,male,1,not_member,6.67,19,White,Other,Manufacturing,Unmarried
3,12,no,male,4,not_member,4.00,22,White,Other,Other,Unmarried
4,12,no,male,17,not_member,7.50,35,White,Other,Other,Married


#### Task

Build regression models to predict `WAGE`.  Incorporate the categorical features and transform the target using a logarithm.  Build `Ridge` models and consider some different amounts of regularization.  

After fitting your model, interpret the model and try to understand what features led to higher wages.  Consider using `permutation_importance` that you encountered in module 8.  Discuss your findings in the class forum.

For an in depth example discussing the perils of interpreting the coefficients see the example in scikitlearn examples [here](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html).

In [28]:
# convert sklearn Bunch to DataFrame
df = wages.frame
# check for nulls
df.isnull().sum()
df.shape

(534, 11)

In [39]:
# Transform target variable using logarithm
df['WAGE'] = np.log1p(df['WAGE'])

# Split df into features (X) and target (y)
X = df.drop(columns=['WAGE'])
y = df['WAGE']

# Split df into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: ColumnTransformer with OneHotEncoder for categorical variables
categorical_features = ['EDUCATION', 'SOUTH', 'SEX', 'UNION', 'RACE', 'OCCUPATION', 'SECTOR', 'MARR']
numeric_features = ['EXPERIENCE', 'AGE']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Define Ridge, LASSO, and Sequential Feature Selection (SFS) pipelines
ridge_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', Ridge())])
lasso_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', Lasso())])
sfs_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', LinearRegression())])

# Define alpha values for Ridge and LASSO
ridge_alphas = [0.01, 0.1, 1, 10, 100]
lasso_alphas = [0.001, 0.01, 0.1, 1, 10]

# Perform model fitting and interpretation
for alpha in ridge_alphas:
    ridge_pipeline.set_params(model__alpha=alpha)
    ridge_pipeline.fit(X_train, y_train)
    y_pred = ridge_pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Ridge with Alpha={alpha}, Mean Squared Error: {mse:.4f}')

for alpha in lasso_alphas:
    lasso_pipeline.set_params(model__alpha=alpha)
    lasso_pipeline.fit(X_train, y_train)
    y_pred = lasso_pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'LASSO with Alpha={alpha}, Mean Squared Error: {mse:.4f}')

# Perform Sequential Feature Selection
sfs = SequentialFeatureSelector(LinearRegression(), n_features_to_select=5, direction='forward', scoring='neg_mean_squared_error')
sfs.fit(preprocessor.fit_transform(X_train), y_train)

print("Selected Features (SFS):", sfs.get_support(indices=True))

Ridge with Alpha=0.01, Mean Squared Error: 0.0004
Ridge with Alpha=0.1, Mean Squared Error: 0.0004
Ridge with Alpha=1, Mean Squared Error: 0.0004
Ridge with Alpha=10, Mean Squared Error: 0.0005
Ridge with Alpha=100, Mean Squared Error: 0.0005
LASSO with Alpha=0.001, Mean Squared Error: 0.0006
LASSO with Alpha=0.01, Mean Squared Error: 0.0007
LASSO with Alpha=0.1, Mean Squared Error: 0.0007
LASSO with Alpha=1, Mean Squared Error: 0.0007
LASSO with Alpha=10, Mean Squared Error: 0.0007
Selected Features (SFS): [ 0  1 22 30 32]
